# Retail Sales Data Analysis - Data Exploration

This notebook provides comprehensive exploratory data analysis (EDA) for retail sales data.

## Objectives
- Load and examine the dataset structure
- Understand data quality and completeness
- Explore distributions and patterns
- Identify potential data issues
- Generate initial insights

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Import custom modules
import sys
sys.path.append('../src')
from data_processing import DataProcessor
from visualization import RetailVisualizer
from analysis import RetailAnalyzer

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Libraries imported successfully!")

## 1. Data Loading and Initial Inspection

In [ ]:
# Initialize data processor
processor = DataProcessor()

# Generate sample data for demonstration
# In real scenario, you would load your actual data:
# data = processor.load_data('path/to/your/data.csv')

data = processor.generate_sample_data(n_customers=1000, n_transactions=5000)
print(f"Dataset loaded with shape: {data.shape}")

In [ ]:
# Display basic information about the dataset
print("Dataset Info:")
print("=" * 50)
data.info()

print("\nFirst 5 rows:")
print("=" * 50)
data.head()

In [ ]:
# Check for missing values
print("Missing Values:")
print("=" * 30)
missing_values = data.isnull().sum()
missing_percentage = (missing_values / len(data)) * 100
missing_df = pd.DataFrame({
    'Missing Count': missing_values,
    'Percentage': missing_percentage
})
print(missing_df[missing_df['Missing Count'] > 0])

if missing_df['Missing Count'].sum() == 0:
    print("No missing values found!")

In [ ]:
# Basic descriptive statistics
print("Descriptive Statistics for Numerical Columns:")
print("=" * 50)
data.describe()

## 2. Data Quality Assessment

In [ ]:
# Check for duplicates
duplicates = data.duplicated().sum()
print(f"Number of duplicate rows: {duplicates}")

# Check unique values in categorical columns
categorical_cols = data.select_dtypes(include=['object']).columns
print("\nUnique values in categorical columns:")
for col in categorical_cols:
    print(f"{col}: {data[col].nunique()} unique values")
    if data[col].nunique() < 20:
        print(f"  Values: {data[col].unique()}")
    print()

In [ ]:
# Check data ranges and potential outliers
numerical_cols = data.select_dtypes(include=[np.number]).columns
print("Data Ranges and Potential Issues:")
print("=" * 40)

for col in numerical_cols:
    if col not in ['customer_id', 'product_id', 'transaction_id']:
        print(f"\n{col}:")
        print(f"  Range: {data[col].min():.2f} to {data[col].max():.2f}")
        print(f"  Mean: {data[col].mean():.2f}")
        print(f"  Std: {data[col].std():.2f}")
        
        # Check for negative values where they shouldn't exist
        if col in ['price', 'quantity', 'total_amount'] and data[col].min() < 0:
            print(f"  WARNING: Negative values found in {col}")
        
        # Check for extreme outliers (beyond 3 standard deviations)
        mean_val = data[col].mean()
        std_val = data[col].std()
        outliers = data[(data[col] < mean_val - 3*std_val) | (data[col] > mean_val + 3*std_val)]
        if len(outliers) > 0:
            print(f"  Extreme outliers: {len(outliers)} ({len(outliers)/len(data)*100:.1f}%)")

## 3. Distribution Analysis

In [ ]:
# Plot distributions of key numerical variables
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('Distribution of Key Variables', fontsize=16, fontweight='bold')

# Total Amount
axes[0, 0].hist(data['total_amount'], bins=30, alpha=0.7, edgecolor='black')
axes[0, 0].set_title('Total Amount Distribution')
axes[0, 0].set_xlabel('Total Amount ($)')
axes[0, 0].set_ylabel('Frequency')

# Price
axes[0, 1].hist(data['price'], bins=30, alpha=0.7, edgecolor='black')
axes[0, 1].set_title('Price Distribution')
axes[0, 1].set_xlabel('Price ($)')
axes[0, 1].set_ylabel('Frequency')

# Quantity
axes[0, 2].hist(data['quantity'], bins=20, alpha=0.7, edgecolor='black')
axes[0, 2].set_title('Quantity Distribution')
axes[0, 2].set_xlabel('Quantity')
axes[0, 2].set_ylabel('Frequency')

# Age
axes[1, 0].hist(data['age'], bins=25, alpha=0.7, edgecolor='black')
axes[1, 0].set_title('Customer Age Distribution')
axes[1, 0].set_xlabel('Age')
axes[1, 0].set_ylabel('Frequency')

# Discount
axes[1, 1].hist(data['discount'], bins=20, alpha=0.7, edgecolor='black')
axes[1, 1].set_title('Discount Distribution')
axes[1, 1].set_xlabel('Discount Rate')
axes[1, 1].set_ylabel('Frequency')

# Box plot for total amount
axes[1, 2].boxplot(data['total_amount'])
axes[1, 2].set_title('Total Amount Box Plot')
axes[1, 2].set_ylabel('Total Amount ($)')

plt.tight_layout()
plt.show()

In [ ]:
# Categorical variable distributions
fig, axes = plt.subplots(2, 2, figsize=(16, 10))
fig.suptitle('Categorical Variable Distributions', fontsize=16, fontweight='bold')

# Category distribution
category_counts = data['category'].value_counts()
axes[0, 0].bar(category_counts.index, category_counts.values)
axes[0, 0].set_title('Product Category Distribution')
axes[0, 0].set_xlabel('Category')
axes[0, 0].set_ylabel('Count')
axes[0, 0].tick_params(axis='x', rotation=45)

# Gender distribution
gender_counts = data['gender'].value_counts()
axes[0, 1].pie(gender_counts.values, labels=gender_counts.index, autopct='%1.1f%%')
axes[0, 1].set_title('Gender Distribution')

# City distribution
city_counts = data['city'].value_counts()
axes[1, 0].bar(city_counts.index, city_counts.values)
axes[1, 0].set_title('City Distribution')
axes[1, 0].set_xlabel('City')
axes[1, 0].set_ylabel('Count')
axes[1, 0].tick_params(axis='x', rotation=45)

# Transaction date distribution (by month)
monthly_transactions = data.groupby(data['transaction_date'].dt.month).size()
month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
               'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
axes[1, 1].bar(range(1, len(monthly_transactions)+1), monthly_transactions.values)
axes[1, 1].set_title('Transactions by Month')
axes[1, 1].set_xlabel('Month')
axes[1, 1].set_ylabel('Number of Transactions')
axes[1, 1].set_xticks(range(1, len(monthly_transactions)+1))
axes[1, 1].set_xticklabels([month_names[i-1] for i in monthly_transactions.index])

plt.tight_layout()
plt.show()

## 4. Correlation Analysis

In [ ]:
# Calculate correlation matrix for numerical variables
numerical_data = data.select_dtypes(include=[np.number])
correlation_matrix = numerical_data.corr()

# Create correlation heatmap
plt.figure(figsize=(12, 10))
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
sns.heatmap(correlation_matrix, mask=mask, annot=True, cmap='coolwarm', center=0,
            square=True, linewidths=0.5, cbar_kws={"shrink": .8})
plt.title('Correlation Matrix of Numerical Variables')
plt.tight_layout()
plt.show()

# Print strong correlations
print("Strong Correlations (|r| > 0.5):")
print("=" * 35)
for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        corr_value = correlation_matrix.iloc[i, j]
        if abs(corr_value) > 0.5:
            print(f"{correlation_matrix.columns[i]} vs {correlation_matrix.columns[j]}: {corr_value:.3f}")

## 5. Time Series Analysis

In [ ]:
# Daily sales trend
daily_sales = data.groupby(data['transaction_date'].dt.date)['total_amount'].sum()

plt.figure(figsize=(15, 6))
plt.plot(daily_sales.index, daily_sales.values, linewidth=1, alpha=0.7)
plt.title('Daily Sales Trend')
plt.xlabel('Date')
plt.ylabel('Total Sales ($)')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Monthly aggregation
monthly_sales = data.groupby(data['transaction_date'].dt.to_period('M'))['total_amount'].sum()
monthly_transactions = data.groupby(data['transaction_date'].dt.to_period('M')).size()

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 10))

# Monthly sales
ax1.plot(monthly_sales.index.astype(str), monthly_sales.values, marker='o', linewidth=2)
ax1.set_title('Monthly Sales Trend')
ax1.set_ylabel('Total Sales ($)')
ax1.grid(True, alpha=0.3)
ax1.tick_params(axis='x', rotation=45)

# Monthly transaction count
ax2.bar(monthly_transactions.index.astype(str), monthly_transactions.values, alpha=0.7)
ax2.set_title('Monthly Transaction Count')
ax2.set_xlabel('Month')
ax2.set_ylabel('Number of Transactions')
ax2.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## 6. Customer Analysis

In [ ]:
# Customer-level analysis
customer_summary = data.groupby('customer_id').agg({
    'total_amount': ['sum', 'mean', 'count'],
    'transaction_date': ['min', 'max'],
    'age': 'first',
    'gender': 'first'
})

# Flatten column names
customer_summary.columns = ['Total_Spent', 'Avg_Order_Value', 'Purchase_Frequency', 
                           'First_Purchase', 'Last_Purchase', 'Age', 'Gender']

print("Customer Summary Statistics:")
print("=" * 30)
print(f"Total unique customers: {len(customer_summary)}")
print(f"Average customer lifetime value: ${customer_summary['Total_Spent'].mean():.2f}")
print(f"Average purchase frequency: {customer_summary['Purchase_Frequency'].mean():.1f}")
print(f"Average order value: ${customer_summary['Avg_Order_Value'].mean():.2f}")

# Customer value distribution
fig, axes = plt.subplots(2, 2, figsize=(16, 10))
fig.suptitle('Customer Analysis', fontsize=16, fontweight='bold')

# Customer lifetime value distribution
axes[0, 0].hist(customer_summary['Total_Spent'], bins=30, alpha=0.7, edgecolor='black')
axes[0, 0].set_title('Customer Lifetime Value Distribution')
axes[0, 0].set_xlabel('Total Spent ($)')
axes[0, 0].set_ylabel('Number of Customers')

# Purchase frequency distribution
axes[0, 1].hist(customer_summary['Purchase_Frequency'], bins=20, alpha=0.7, edgecolor='black')
axes[0, 1].set_title('Purchase Frequency Distribution')
axes[0, 1].set_xlabel('Number of Purchases')
axes[0, 1].set_ylabel('Number of Customers')

# Age vs Total Spent scatter plot
axes[1, 0].scatter(customer_summary['Age'], customer_summary['Total_Spent'], alpha=0.6)
axes[1, 0].set_title('Age vs Total Spent')
axes[1, 0].set_xlabel('Age')
axes[1, 0].set_ylabel('Total Spent ($)')

# Gender comparison
gender_spending = customer_summary.groupby('Gender')['Total_Spent'].mean()
axes[1, 1].bar(gender_spending.index, gender_spending.values)
axes[1, 1].set_title('Average Spending by Gender')
axes[1, 1].set_xlabel('Gender')
axes[1, 1].set_ylabel('Average Total Spent ($)')

plt.tight_layout()
plt.show()

## 7. Product Analysis

In [ ]:
# Product performance analysis
product_performance = data.groupby(['product_id', 'category']).agg({
    'total_amount': ['sum', 'count'],
    'quantity': 'sum',
    'price': 'first'
}).round(2)

product_performance.columns = ['Total_Revenue', 'Total_Orders', 'Total_Quantity', 'Unit_Price']
product_performance = product_performance.reset_index()

# Category analysis
category_analysis = data.groupby('category').agg({
    'total_amount': ['sum', 'mean', 'count'],
    'quantity': 'sum',
    'customer_id': 'nunique'
}).round(2)

category_analysis.columns = ['Total_Revenue', 'Avg_Order_Value', 'Total_Orders', 
                           'Total_Quantity', 'Unique_Customers']

print("Category Performance:")
print("=" * 20)
print(category_analysis.sort_values('Total_Revenue', ascending=False))

# Visualizations
fig, axes = plt.subplots(2, 2, figsize=(16, 10))
fig.suptitle('Product Analysis', fontsize=16, fontweight='bold')

# Revenue by category
category_revenue = category_analysis['Total_Revenue'].sort_values(ascending=False)
axes[0, 0].bar(category_revenue.index, category_revenue.values)
axes[0, 0].set_title('Revenue by Category')
axes[0, 0].set_xlabel('Category')
axes[0, 0].set_ylabel('Total Revenue ($)')
axes[0, 0].tick_params(axis='x', rotation=45)

# Average order value by category
category_aov = category_analysis['Avg_Order_Value'].sort_values(ascending=False)
axes[0, 1].bar(category_aov.index, category_aov.values)
axes[0, 1].set_title('Average Order Value by Category')
axes[0, 1].set_xlabel('Category')
axes[0, 1].set_ylabel('Average Order Value ($)')
axes[0, 1].tick_params(axis='x', rotation=45)

# Top 10 products by revenue
top_products = product_performance.nlargest(10, 'Total_Revenue')
axes[1, 0].barh(range(len(top_products)), top_products['Total_Revenue'])
axes[1, 0].set_title('Top 10 Products by Revenue')
axes[1, 0].set_xlabel('Total Revenue ($)')
axes[1, 0].set_ylabel('Product ID')
axes[1, 0].set_yticks(range(len(top_products)))
axes[1, 0].set_yticklabels(top_products['product_id'])

# Price vs Revenue scatter
axes[1, 1].scatter(product_performance['Unit_Price'], product_performance['Total_Revenue'], alpha=0.6)
axes[1, 1].set_title('Unit Price vs Total Revenue')
axes[1, 1].set_xlabel('Unit Price ($)')
axes[1, 1].set_ylabel('Total Revenue ($)')

plt.tight_layout()
plt.show()

## 8. Key Insights and Findings

In [ ]:
# Generate key insights
print("KEY INSIGHTS FROM DATA EXPLORATION")
print("=" * 50)

# Dataset overview
print(f"\n📊 DATASET OVERVIEW:")
print(f"   • Total transactions: {len(data):,}")
print(f"   • Unique customers: {data['customer_id'].nunique():,}")
print(f"   • Unique products: {data['product_id'].nunique():,}")
print(f"   • Date range: {data['transaction_date'].min().date()} to {data['transaction_date'].max().date()}")

# Financial metrics
total_revenue = data['total_amount'].sum()
avg_order_value = data['total_amount'].mean()
print(f"\n💰 FINANCIAL METRICS:")
print(f"   • Total revenue: ${total_revenue:,.2f}")
print(f"   • Average order value: ${avg_order_value:.2f}")
print(f"   • Revenue per customer: ${total_revenue/data['customer_id'].nunique():.2f}")

# Top performing category
top_category = category_analysis['Total_Revenue'].idxmax()
top_category_revenue = category_analysis.loc[top_category, 'Total_Revenue']
print(f"\n🏆 TOP PERFORMERS:")
print(f"   • Best category: {top_category} (${top_category_revenue:,.2f})")
print(f"   • Category market share: {top_category_revenue/total_revenue*100:.1f}%")

# Customer insights
avg_customer_value = customer_summary['Total_Spent'].mean()
avg_purchase_freq = customer_summary['Purchase_Frequency'].mean()
print(f"\n👥 CUSTOMER INSIGHTS:")
print(f"   • Average customer lifetime value: ${avg_customer_value:.2f}")
print(f"   • Average purchase frequency: {avg_purchase_freq:.1f} orders")
print(f"   • Customer age range: {data['age'].min()} - {data['age'].max()} years")

# Seasonal patterns
monthly_sales = data.groupby(data['transaction_date'].dt.month)['total_amount'].sum()
best_month = monthly_sales.idxmax()
worst_month = monthly_sales.idxmin()
month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
               'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
print(f"\n📅 SEASONAL PATTERNS:")
print(f"   • Best performing month: {month_names[best_month-1]} (${monthly_sales[best_month]:,.2f})")
print(f"   • Lowest performing month: {month_names[worst_month-1]} (${monthly_sales[worst_month]:,.2f})")

# Data quality
print(f"\n✅ DATA QUALITY:")
print(f"   • Missing values: {data.isnull().sum().sum()}")
print(f"   • Duplicate rows: {data.duplicated().sum()}")
print(f"   • Data completeness: {(1 - data.isnull().sum().sum()/(len(data)*len(data.columns)))*100:.1f}%")

print(f"\n🎯 NEXT STEPS:")
print(f"   • Proceed to data cleaning and preprocessing")
print(f"   • Perform customer segmentation analysis")
print(f"   • Build predictive models for sales forecasting")
print(f"   • Develop business recommendations")

## 9. Data Export for Further Analysis

In [ ]:
# Save exploration results
# Create summary statistics file
summary_stats = {
    'dataset_overview': {
        'total_transactions': len(data),
        'unique_customers': data['customer_id'].nunique(),
        'unique_products': data['product_id'].nunique(),
        'date_range': f"{data['transaction_date'].min().date()} to {data['transaction_date'].max().date()}"
    },
    'financial_metrics': {
        'total_revenue': float(data['total_amount'].sum()),
        'average_order_value': float(data['total_amount'].mean()),
        'revenue_per_customer': float(data['total_amount'].sum() / data['customer_id'].nunique())
    },
    'data_quality': {
        'missing_values': int(data.isnull().sum().sum()),
        'duplicate_rows': int(data.duplicated().sum()),
        'completeness_percentage': float((1 - data.isnull().sum().sum()/(len(data)*len(data.columns)))*100)
    }
}

# Save customer summary
customer_summary.to_csv('../data/processed/customer_summary.csv')
print("Customer summary saved to ../data/processed/customer_summary.csv")

# Save category analysis
category_analysis.to_csv('../data/processed/category_analysis.csv')
print("Category analysis saved to ../data/processed/category_analysis.csv")

print("\nData exploration completed successfully!")
print("Ready to proceed to data cleaning and preprocessing.")